In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Data Reading

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksstorageadls.dfs.core.windows.net/products")


In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")

### Functions

In [0]:
df.createOrReplaceTempView("products")
# df = df.withColumn("price", col("price").cast(DoubleType()))
# df.printSchema()

In [0]:
%sql
create or replace function databricks_cata.bronze.discount_func(p_price double)
returns double
language sql
return p_price * 0.9


In [0]:
%sql
select product_id, price,databricks_cata.bronze.discount_func(price) as price_discount
from products

In [0]:
df = df.withColumn("discountedprice", expr("databricks_cata.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
create or replace function databricks_cata.bronze.upperfunc(p_brand string)
returns string
language python
as
$$
  return p_brand.upper()
$$

In [0]:
%sql
select product_id, brand, databricks_cata.bronze.upperfunc(brand) as brand_upper
from products

In [0]:
df.write.format("delta").mode("overwrite")\
      .option("path", "abfss://silver@databricksstorageadls.dfs.core.windows.net/products")\
          .save()

In [0]:
%sql
create table if not exists databricks_cata.silver.products_silver
using delta
location "abfss://silver@databricksstorageadls.dfs.core.windows.net/products"

In [0]:
%sql
select * from databricks_cata.silver.products_silver